In [9]:
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier


## Defining the bucket 
s3 = boto3.resource('s3')
bucket_name = 'data-445'
bucket = s3.Bucket(bucket_name)

## Defining the csv file 
file_key_1 = 'Demos/churn-bigml-80.csv'
file_key_2 = 'Demos/churn-bigml-20.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

## Reading the csv files
telecom_train = pd.read_csv(file_content_stream_1)
telecom_test = pd.read_csv(file_content_stream_2)

In [10]:
## Changing logical value to numbers
telecom_train['Churn_numb'] = np.where(telecom_train['Churn'] == False, 0, 1)
telecom_test['Churn_numb'] = np.where(telecom_test['Churn'] == False, 0, 1)

telecom_train['International_plan'] = np.where(telecom_train['International_plan'] == 'No', 0, 1)
telecom_test['International_plan'] = np.where(telecom_test['International_plan'] == 'No', 0, 1)

telecom_train['Voice_mail_plan'] = np.where(telecom_train['Voice_mail_plan'] == 'No', 0, 1)
telecom_test['Voice_mail_plan'] = np.where(telecom_test['Voice_mail_plan'] == 'No', 0, 1)

telecom_train['total_charge'] = telecom_train['Total_day_charge'] + telecom_train['Total_eve_charge'] + telecom_train['Total_night_charge'] + telecom_train['Total_intl_charge']
telecom_test['total_charge'] = telecom_test['Total_day_charge'] + telecom_test['Total_eve_charge'] + telecom_test['Total_night_charge'] + telecom_test['Total_intl_charge']

telecom_train.head()

,State,Account_length,Area_code,International_plan,Voice_mail_plan,Number_vmail_messages,Total_day_minutes,Total_day_calls,Total_day_charge,Total_eve_minutes,...,Total_night_minutes,Total_night_calls,Total_night_charge,Total_intl_minutes,Total_intl_calls,Total_intl_charge,Customer_service_calls,Churn,Churn_numb,total_charge
0,KS,128,415,0,1,25,265.1,110,45.07,197.4,...,244.7,91,11.01,10.0,3,2.70,1,False,0,75.56
1,OH,107,415,0,1,26,161.6,123,27.47,195.5,...,254.4,103,11.45,13.7,3,3.70,1,False,0,59.24
2,NJ,137,415,0,0,0,243.4,114,41.38,121.2,...,162.6,104,7.32,12.2,5,3.29,0,False,0,62.29
3,OH,84,408,1,0,0,299.4,71,50.90,61.9,...,196.9,89,8.86,6.6,7,1.78,2,False,0,66.80
4,OK,75,415,1,0,0,166.7,113,28.34,148.3,...,186.9,121,8.41,10.1,3,2.73,3,False,0,52.09


In [11]:
## Selecting variables of interest
telecom_train = telecom_train[['Account_length', 'International_plan', 'Voice_mail_plan', 'total_charge', 'Customer_service_calls', 'Churn_numb']]
telecom_test = telecom_test[['Account_length', 'International_plan', 'Voice_mail_plan', 'total_charge', 'Customer_service_calls', 'Churn_numb']]

telecom_train.head()

,Account_length,International_plan,Voice_mail_plan,total_charge,Customer_service_calls,Churn_numb
0,128,0,1,75.56,1,0
1,107,0,1,59.24,1,0
2,137,0,0,62.29,0,0
3,84,1,0,66.80,2,0
4,75,1,0,52.09,3,0


In [15]:
## Defining the input and target variables
X = telecom_train[['Account_length', 'International_plan', 'Voice_mail_plan', 'total_charge', 'Customer_service_calls']]
Y = telecom_train['Churn_numb']

## Defining data-frame to store results 
RF_importances = pd.DataFrame({'Account_length': np.repeat(np.nan, 100), 'International_plan': np.repeat(np.nan, 100), 'Voice_mail_plan': np.repeat(np.nan, 100), 'total_charge': np.repeat(np.nan, 100), 'Customer_service_calls': np.repeat(np.nan, 100)})
Ada_importances = pd.DataFrame({'Account_length': np.repeat(np.nan, 100), 'International_plan': np.repeat(np.nan, 100), 'Voice_mail_plan': np.repeat(np.nan, 100), 'total_charge': np.repeat(np.nan, 100), 'Customer_service_calls': np.repeat(np.nan, 100)})
GB_importances = pd.DataFrame({'Account_length': np.repeat(np.nan, 100), 'International_plan': np.repeat(np.nan, 100), 'Voice_mail_plan': np.repeat(np.nan, 100), 'total_charge': np.repeat(np.nan, 100), 'Customer_service_calls': np.repeat(np.nan, 100)})

for i in range(0, 10):
    
    ## Splitting the data 
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)
    
    ###################
    ## Random Forest ##
    ###################
    
    RF = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train, Y_train)
    RF_importances.loc[i] = RF.feature_importances_
    
    ##############
    ## AdaBoost ##
    ##############
    
    Ada = AdaBoostClassifier(n_estimators = 500, base_estimator = DecisionTreeClassifier(max_depth = 3), learning_rate = 0.01).fit(X_train, Y_train)
    Ada_importances.loc[i] = Ada.feature_importances_
    
    #######################
    ## Gradient Boosting ##
    #######################
    
    GB = GradientBoostingClassifier(n_estimators = 500, max_depth = 3, learning_rate = 0.01).fit(X_train, Y_train)
    GB_importances.loc[i] = GB.feature_importances_

TypeError: __init__() got an unexpected keyword argument 'max_depth'

In [14]:
RF_importances.head(20)

,Account_length,International_plan,Voice_mail_plan,total_charge,Customer_service_calls
0,0.021501,0.164912,0.076438,0.523910,0.213239
1,0.017824,0.138211,0.075256,0.582717,0.185993
2,0.020646,0.126847,0.085638,0.555742,0.211126
3,0.019404,0.126673,0.087818,0.566279,0.199826
4,0.020154,0.124344,0.075509,0.560639,0.219353
5,0.019866,0.159662,0.083450,0.552658,0.184364
6,0.016651,0.132013,0.078098,0.560024,0.213214
7,0.017400,0.124300,0.070614,0.566932,0.220754
8,0.021266,0.152808,0.074244,0.557451,0.194231
9,0.017968,0.132537,0.076671,0.582744,0.190081
